In [ ]:
pip install finance-datareader

In [ ]:
pip install pykrx

In [ ]:
import pandas as pd
import itertools
import numpy as np
import FinanceDataReader as fdr
from concurrent.futures import ThreadPoolExecutor
from pykrx import stock
from datetime import datetime
from google.colab import files

In [ ]:
# 파일 업로드
uploaded = files.upload()  # 여러 개의 파일을 업로드 가능

# 업로드한 파일 리스트 확인
file_list = list(uploaded.keys())

Saving 투자경고.xlsx to 투자경고.xlsx


In [ ]:
# 각 파일을 pandas DataFrame으로 읽기 (engine="openpyxl" 추가)
df1 = pd.read_excel(file_list[0], engine="openpyxl")
df2=df1.iloc[:,1:]

In [ ]:
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

# 날짜 데이터 변환
df2["해제일"] = pd.to_datetime(df2["해제일"])
# 주가 데이터 불러오기 함수
def get_stock_data(code, release_date):
    start_date = (release_date - pd.DateOffset(days=30)).strftime('%Y-%m-%d')
    end_date = (release_date + pd.DateOffset(days=40)).strftime('%Y-%m-%d')

    try:
        # 주가 데이터 불러오기
        stock_df = fdr.DataReader(code, start_date, end_date)

        future_dates = {
            "5일후": release_date + pd.DateOffset(days=5),
            "10일후": release_date + pd.DateOffset(days=10),
            "30일후": release_date + pd.DateOffset(days=30),
        }

        stock_prices = {}
        release_price = None

        # 해제일의 주가 가져오기
        if release_date.strftime('%Y-%m-%d') in stock_df.index:
            release_price = stock_df.loc[release_date.strftime('%Y-%m-%d'), "Close"]

        # 5일후, 10일후, 30일후의 주가 가져오기
        for key, date in future_dates.items():
            if date.strftime('%Y-%m-%d') in stock_df.index:
                stock_prices[key] = stock_df.loc[date.strftime('%Y-%m-%d'), "Close"]
            else:
                stock_prices[key] = None

        # 해제일 주가 추가
        stock_prices["해제일 주가"] = release_price

        return stock_prices
    except Exception as e:
        print(f"Error fetching data for {code}: {e}")
        return {"5일후": None, "10일후": None, "30일후": None, "해제일 주가": None}

# 병렬 처리로 주가 데이터 가져오기
def get_stock_data_parallel(df):
    with ThreadPoolExecutor(max_workers=10) as executor:
        results = list(executor.map(lambda row: get_stock_data(row.종목코드, row.해제일), df.itertuples()))

    # 결과를 DataFrame으로 변환
    stock_df = pd.DataFrame(results)
    df[["5일후", "10일후", "30일후", "해제일 주가"]] = stock_df

    return df

# 모든 종목에 대해 병렬로 주가 데이터 가져오기
df_stock_data = get_stock_data_parallel(df2)

# 수익률 계산 함수
def calculate_returns(row):
    # 해제일 주가는 "해제일 주가"로 가져온 주가입니다.
    release_price = row["해제일 주가"]

    if row["5일후"] is not None and release_price is not None:
        row["5일수익률"] = (row["5일후"] - release_price) / release_price
    else:
        row["5일수익률"] = None

    if row["10일후"] is not None and release_price is not None:
        row["10일수익률"] = (row["10일후"] - release_price) / release_price
    else:
        row["10일수익률"] = None

    if row["30일후"] is not None and release_price is not None:
        row["30일수익률"] = (row["30일후"] - release_price) / release_price
    else:
        row["30일수익률"] = None

    return row

# 수익률 계산
df_stock_data = df_stock_data.apply(calculate_returns, axis=1)

# 결과 출력
print(df_stock_data)


           종목명    종목코드        공시일        지정일        해제일       5일후      10일후  \
0          두산우  000155 2025-02-21 2025-02-24 2025-03-11       NaN       NaN   
1         현대로템  064350 2025-02-21 2025-02-24 2025-03-12       NaN       NaN   
2       한국무브넥스  010100 2025-02-05 2025-02-06 2025-02-20    4940.0       NaN   
3         가온전선  000500 2025-01-24 2025-01-31 2025-02-14   57900.0   53600.0   
4           두산  000150 2025-01-24 2025-01-31 2025-02-14  357000.0  366000.0   
...        ...     ...        ...        ...        ...       ...       ...   
1107  SH에너지화학우  002365 2007-09-03 2007-09-04 2007-09-18       NaN   39500.0   
1108       신원우  009275 2007-09-03 2007-09-04 2007-09-18       NaN    6300.0   
1109     아남전자우  008705 2007-09-03 2007-09-04 2007-09-18       NaN   16600.0   
1110   현대금속2우B  018417 2007-09-03 2007-09-04 2007-09-14   10050.0       NaN   
1111     현대금속우  018415 2007-09-03 2007-09-04 2007-09-12  734000.0       NaN   

          30일후    해제일 주가     5일수익률    10일수익률    30일

In [ ]:
df_returns_filt = df_stock_data[~df_stock_data["종목코드"].str.endswith(("5", "7"))]

In [ ]:
def get_market_cap_on_date(stock_code, release_date):
    """특정 날짜의 특정 종목의 시가총액을 조회하는 함수"""
    try:
        # 날짜를 기준으로 시가총액을 조회 (ticker 인자 추가)
        market_cap_data = stock.get_market_cap_by_date(release_date, release_date, ticker=stock_code)

        # 시가총액 값이 있으면 반환
        if not market_cap_data.empty:
            return market_cap_data['시가총액'].values[0]
        else:
            print(f"No market cap data for {stock_code} on {release_date}")
            return None
    except Exception as e:
        print(f"Error for {stock_code} on {release_date}: {e}")
        return None


# '해제일'을 datetime 형식으로 변환
df_returns_filt["해제일"] = pd.to_datetime(df_returns_filt["해제일"])

# 각 종목에 대해 해제일 기준 시가총액 조회
df_returns_filt["시가총액"] = df_returns_filt.apply(
    lambda row: get_market_cap_on_date(row["종목코드"], row["해제일"]), axis=1
)

# 결과 출력
print(df_returns_filt)

In [ ]:
# 해제일 기준으로 특정 기간 필터링하는 함수
def filter_by_date_range(df, start_date, end_date):
    df['해제일'] = pd.to_datetime(df['해제일'])  # '해제일'을 datetime으로 변환
    filtered_df = df[(df['해제일'] >= start_date) & (df['해제일'] <= end_date)]
    return filtered_df

# 예시: 해제일 기준으로 2022년 이후
df_filtered_2020 = filter_by_date_range(df_returns_filt, '2020-03-13', '2021-05-03')

# 예시: 해제일 기준으로 2020-2021년
df_filtered_2023 = filter_by_date_range(df_returns_filt, '2023-11-05', '2025-02-28')

# 예시: 해제일 기준으로 2024년 9월 ~ 2025년 2월
df_filtered_2008 = filter_by_date_range(df_returns_filt, '2008-10-01', '2009-06-01')

# 시가총액 범주 설정 함수
def categorize_market_cap(market_cap):
    if market_cap < 3e10:
        return "Small Cap"  # 시가총액이 3,000억 미만
    elif market_cap < 1e11:
        return "Mid Cap"    # 시가총액이 3,000억 이상 ~ 1조 미만
    else:
        return "Large Cap"  # 시가총액이 10조 이상

# 시가총액 범주 컬럼 추가
df_filtered_2020['시가총액범주'] = df_filtered_2020['시가총액'].apply(categorize_market_cap)
df_filtered_2023['시가총액범주'] = df_filtered_2023['시가총액'].apply(categorize_market_cap)
df_filtered_2008['시가총액범주'] = df_filtered_2008['시가총액'].apply(categorize_market_cap)

# 각 기간별 시가총액 범주별로 5일, 10일, 30일 수익률 평균 계산
market_cap_avg_returns_2020 = df_filtered_2020.groupby('시가총액범주')[['5일수익률', '10일수익률', '30일수익률']].mean()
market_cap_avg_returns_2023 = df_filtered_2023.groupby('시가총액범주')[['5일수익률', '10일수익률', '30일수익률']].mean()
market_cap_avg_returns_2008 = df_filtered_2008.groupby('시가총액범주')[['5일수익률', '10일수익률', '30일수익률']].mean()

# 결과 출력
print("코로나 공매도 금지")
print(market_cap_avg_returns_2020)
print("23년 무차입공매도금지")
print(market_cap_avg_returns_2023)
print("서브프라임 공매도 금지")
print(market_cap_avg_returns_2008)


In [ ]:
market_cap_avg_returns_2020

,5일수익률,10일수익률,30일수익률
시가총액범주,,,
Large Cap,-0.018168,0.044642,0.088129
Mid Cap,0.003394,0.181621,0.223888
Small Cap,-0.005697,-0.072553,-0.088439


In [ ]:
market_cap_avg_returns_2023

,5일수익률,10일수익률,30일수익률
시가총액범주,,,
Large Cap,0.026642,-0.008013,0.019180
Mid Cap,-0.074957,-0.010201,-0.102435
Small Cap,0.117910,-0.011961,-0.124296


In [ ]:
market_cap_avg_returns_2008

,5일수익률,10일수익률,30일수익률
시가총액범주,,,
Large Cap,-0.052527,0.012747,0.098461
Mid Cap,-0.061691,-0.047295,-0.054653
Small Cap,0.060957,0.057285,-0.118452
